# Tutorial: Perturbation of the Neoclassical Growth Model

## Advanced Macro: numerical methods

__Warm-up: install and test the `ForwardDiff` library. Check the jacobian function.__

In [1]:
function f(x::Vector{Float64})::Vector{Float64}
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

f (generic function with 1 method)

In [ ]:
f([0.1, 0.5])

In [ ]:
# option 1
import ForwardDiff
ForwardDiff.jacobian

In [ ]:
# option 2
using ForwardDiff: jacobian

In [ ]:
jacobian(f, [0.1, 0.3])

In [ ]:
This doesn't work because ForwardDiff tries to pass Dual Numbers as arguments to f.
We need to loosen the definition of f, so that it takes vectors of dual numbers.

In [ ]:
# function f(x::Vector{Any})::Vector{Number}

function f(x)
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

In [ ]:
jacobian(f, [0.1, 0.3])

__Warm-up(2): install and test the `NLSolve` library. Find the root of a simple 2 variables function.__

In [ ]:
import NLsolve

In [ ]:
function fun(x)
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b) - 0.1
    return [x1,x2]
end

In [ ]:
methods(NLsolve.nlsolve)

In [ ]:
@time NLsolve.nlsolve(fun, [0.0, 0.0])

In [ ]:
@time NLsolve.nlsolve(fun, [0.0, 0.0]; method=:newton)

__Create a structure `Calibration` to hold the model parameters.__

In [ ]:
module Temp
    struct T
        a
    end
end

In [1]:
struct Calibration
    α
    β
    γ
    δ
    ρ
end

In [2]:
p0 = Calibration(
    0.3,  # capital share (from the data)
    0.96, # time discount  (calibrated to 1/(1+r), where r=4%)
    4.0,  # risk aversion or 1/EIS. EIS is calibrated from macro models
    0.1,   # capital depreciation (from the data)
    0.9
)

Calibration(0.3, 0.96, 4.0, 0.1, 0.9)

In [3]:
# Define a constructor with default values

Calibration(; α=0.3, β=0.96, γ=4.0, δ=0.1, ρ=0.9) = Calibration(α, β, γ, δ, ρ)

Calibration

In [4]:
Calibration(; β=0.8)

Calibration(0.3, 0.8, 4.0, 0.1, 0.9)

__Define two functions:__
- `transition(z::Float64, k::Float64, i::Float64, p::Calibration)::Tuple{Float64}` which returns productivity and capital at date `t+1` as a function of productivity, capital and investment at date `t`
- `arbitrage(z::Float64, k::Float64, i::Float64, Z::Float64, K::Float64, I::Float64, p::Calibration)::Float64` which returns the residual of the euler equation (lower case variable for date t, upper case for date t+1)


In [5]:
function transition(z, k, i,  p)
    Z = p.ρ * z
    K = (1-p.δ)*k + i
    return [Z, K]
end

transition (generic function with 1 method)

In [6]:
function arbitrage(z, k, i, Z, K, I, p)
    
    # at date t
    y = exp(z)*k^p.α
    c = y - i

    # at date t+1
    Y = exp(Z)*K^p.α
    C = Y - I

    optimality = p.β*(C/c)^(-p.γ)*(1-p.δ + p.α*exp(Z)*K^(p.α-1) ) - 1

    return [ optimality ]

end

arbitrage (generic function with 1 method)

__Using multiple dispatch, define two variants of the same functions, that take vectors as input and output arguments:__
- `arbitrage(s::Vector{Float64}, x::Vector{Float64}, S::Vector{Float64}, X::Vector{Float64}, p::Calibration)`
- `transition(s::Vector{Float64}, x::Vector{Float64}, p::Calibration)`

In [7]:
transition(s, x, p) = transition(s[1],s[2],x[1],p)
arbitrage(s, x, S, X, p) = arbitrage(s[1],s[2],x[1],S[1],S[2],X[1],p)

arbitrage (generic function with 2 methods)

In [8]:
# let's check it works:
transition([0.1, 4.0], [0.4], p0)

arbitrage([0.1, 4.0], [0.4], [0.1, 4.0], [0.4], p0)

1-element Vector{Float64}:
 -0.015390937406600913

__Write a function `steady_state(p::Calibration)::Tuple{Vector,Vector}` which computes the steady-state of the model computed by hand.__ It returns two vectors, one for the states, one for the controls. Check that the steady-state satisfies the model equations.


In [9]:
function steady_state(p::Calibration)

    K = ((1/p.β-(1-p.δ))/p.α)^(1/(p.α-1))
    i = K*p.δ
    z = 0.0


    s = [z,K]
    x = [i]

    return s, x
end

steady_state (generic function with 1 method)

In [10]:
s, x = steady_state(p0)

([0.0, 2.9208221499640707], [0.2920822149964071])

In [11]:
arbitrage(s,x,s,x,p0)

1-element Vector{Float64}:
 0.0

The first order system satisfies:
$$\begin{align}A s_t + B x_t + C s_{t+1} + D x_{t+1} & = & 0 \\\\ 
s_{t+1} & = & E s_t + F x_t
 \end{align}$$

__Define a structure `PerturbedModel` to hold matrices A,B,C,D,E,F.__



In [12]:
using ForwardDiff: jacobian

In [13]:
A = jacobian( u->arbitrage(u, x, s, x, p0), s )
B = jacobian( u->arbitrage(s, u, s, x, p0), x )
C = jacobian( u->arbitrage(s, x, u, x, p0), s )
D = jacobian( u->arbitrage(s, x, s, u, p0), x )
E = jacobian( u->transition(u, x, p0), s )
F = jacobian( u->transition(s, u, p0), x )

2×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0

In [14]:
struct PerturbedModel
    s::Vector{Float64}
    x::Vector{Float64}
    A::Matrix{Float64}
    B::Matrix{Float64}
    C::Matrix{Float64}
    D::Matrix{Float64}
    E::Matrix{Float64}
    F::Matrix{Float64}
end

In [29]:
function PerturbedModel(p0::Calibration)
    s, x = steady_state(p0)
    A = jacobian( u->arbitrage(u, x, s, x, p0), s )
    B = jacobian( u->arbitrage(s, u, s, x, p0), x )
    C = jacobian( u->arbitrage(s, x, u, x, p0), s )
    D = jacobian( u->arbitrage(s, x, s, u, p0), x )
    E = jacobian( u->transition(u, x, p0), s )
    F = jacobian( u->transition(s, u, p0), x )
    return PerturbedModel(
        s,x,A,B,C,D,E,F
    )

end

PerturbedModel

In [30]:
m = PerturbedModel(p0)

PerturbedModel([0.0, 2.9208221499640707], [0.2920822149964071], [5.074626865671642 0.5212190203776083], [-3.679193085018409;;], [-4.938626865671642 -0.5538125831185547], [3.679193085018409;;], [0.9 0.0; 0.0 0.9], [0.0; 1.0;;])

In [31]:
m.A

1×2 Matrix{Float64}:
 5.07463  0.521219

__Write a function `first_order_model(s::Vector, x::Vector, p::Calibration)::PerturbedModel`, which returns the first order model, given the steady-state and the calibration. Suggestion: use `ForwardDiff.jl` library.__

In [32]:
first_order_model(p) = PerturbedModel(p)

first_order_model (generic function with 1 method)

__We look for a linear solution $x_t = X s_t$ . Write the matrix equation which `X` must satisfy. Write a function `residual(X::Array, M::PerturbedModel)` which computes the residual of this equation for a given `X`.__


In [33]:
function residual(X::Matrix{Float64}, M::PerturbedModel)
    Y = X
    A = M.A
    B = M.B
    C = M.C
    D = M.D
    E = M.E
    F = M.F
    R = A+B*X+C*(E+F*X)+D*Y*(E+F*X)
    return R
end

residual (generic function with 1 method)

In [34]:
X0 = [0.1 -0.1;]

1×2 Matrix{Float64}:
 0.1  -0.1

In [36]:
residual(X0, m)

1×2 Matrix{Float64}:
 0.500898  0.151753

__Write a function `T(X, M::PerturbedModel)`  which implements the time iteration step.__

In [47]:
A = m.A
B = m.B
C = m.C
D = m.D
E = m.E
F = m.F
for mat in [A,B,C,D,E,F]
    println(size(mat))
end

(1, 2)
(1, 1)
(1, 2)
(1, 1)
(2, 2)
(2, 1)


In [48]:
X0

1×2 Matrix{Float64}:
 0.1  -0.1

In [49]:
function T(Y, M::PerturbedModel)
    A = M.A
    B = M.B
    C = M.C
    D = M.D
    E = M.E
    F = M.F
    rhs = (A+(C+D*Y)*E)
    lhs = (B+(C+D*Y)*F)
    return -lhs\rhs
end

T (generic function with 1 method)

In [50]:
T(X0, m)

1×2 Matrix{Float64}:
 0.208869  -0.0670169

__Write function `linear_time_iteration(X_0::Matrix, m::PerturbedModel)::Matrix` which implements the time iteration algorithm. Apply it to `X0 = rand(1,2)` and check that the result satisfies the first order model.__

    

In [56]:
function linear_time_iteration(X_0, m; maxit=100, tol_η=1e-8, verbose=false)
    X0 = X_0
    X = X0
    for i=1:100
        X = T(X0, m)
        η = maximum(abs, X-X0)
        if η<tol_η
            break
        end
        if verbose
            println(i, " : ", η)
        end
        X0 = X
    end
    return X0
end


linear_time_iteration (generic function with 1 method)

In [61]:
@time X = linear_time_iteration(X0, m; verbose=false)

  0.000155 seconds (1.12 k allocations: 81.375 KiB)


1×2 Matrix{Float64}:
 0.768674  0.0278097

In [59]:
X = linear_time_iteration(X0, m; verbose=true)

1 : 0.10886888377842455
2 : 0.08613345778233517
3 : 0.07044233243348069
4 : 0.05878936923161998
5 : 0.0496871401495147
6 : 0.04232583621093261
7 : 0.03622904486714562
8 : 0.031097262949099314
9 : 0.026730763968567106
10 : 0.02298907534219563
11 : 0.019768569738843222
12 : 0.016989542377253475
13 : 0.014588491117756952
14 : 0.012513359697217563
15 : 0.0107205208790222
16 : 0.009172806781953091
17 : 0.00783818227662958
18 : 0.006688820079616575
19 : 0.005700430751929941
20 : 0.004851757154337832
21 : 0.004124177142392549
22 : 0.0035013793772646906
23 : 0.0029690902433593402
24 : 0.0025148380491628597
25 : 0.0021277457900206187
26 : 0.0017983469118587614
27 : 0.0015184204501008658
28 : 0.001280843083745542
29 : 0.0010794563319579398
30 : 0.0009089475160325433
31 : 0.0007647433318708208
32 : 0.0006429150019541252
33 : 0.0005400940473387905
34 : 0.00045339776709096835
35 : 0.00038036355061255023
36 : 0.0003188911857987531
37 : 0.00026719236667449575
38 : 0.0002237466491372686
39 : 0.0001872

1×2 Matrix{Float64}:
 0.768674  0.0278097

In [77]:
residual(X, m)

1×2 Matrix{Float64}:
 3.47455e-8  1.33212e-9

__Define two linear operators `L_S(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` and `L_T(S::Matrix, X_0::Matrix, m::PerturbedModel)::Matrix` which implement the derivatives of the simulation and the time-iteration operator respectively.__

In [67]:
L_S(ΔS,X,m) = (m.E + m.F*X)*ΔS

function L_T(ΔY, X,Y, m) 
    - (B+(C+D*Y)*F)\(D*ΔY*(E+F*X))
end


L_T (generic function with 2 methods)

In [79]:
L_T(2*X, X, X,m) - 2*L_T(X, X, X,m)

1×2 Matrix{Float64}:
 0.0  0.0

__Implement a function `spectral_radius(f::Function)::Float64` which implements the power iteration method to compute the biggest eigenvalues of the two previously defined operators. Check that Blamnchard-Kahn conditions are met.__

In [73]:
norm(x) = maximum(abs, x)

norm (generic function with 1 method)

In [87]:
function spectral_radius(f, X0; maxit=100)
    x0 = rand(size(X0)...)
    u0 = x0 / norm(x0)
    λ = 1.0

    for it=1:maxit
        x1 = f(u0)
        λ = norm(x1)
        println(λ)
        u0 = x1 / λ
    end

    return λ

end

spectral_radius (generic function with 1 method)

In [91]:
spectral_radius(u->L_S(u, X, m), m.s)

1.1610481510375232
1.1086072084230212
1.0745864500379279
1.0507398557507768
1.033104215643956
1.019538168609045
1.0087832500491263
1.0000513811834968
0.9928238752170649
0.9867453900263912
0.9815643198479171
0.9770975144112072
0.9732084799205365
0.9697934113175426
0.9667719589702287
0.964080957685111
0.9616700655203997
0.9594986666204954
0.9575336303511233
0.955747662695045
0.9541180749901054
0.9526258517416164
0.9512549360538716
0.9499916756302723
0.9488243887673399
0.9477430210773227
0.946738871556617
0.9458043721890109
0.9449329092627238
0.9441186774720756
0.9433565599942138
0.9426420293011821
0.9419710646417353
0.9413400830135007
0.9407458811207333
0.9401855863306711
0.9396566150418812
0.9391566371898438
0.9386835458595643
0.9382354311680174
0.9378105577324763
0.9374073451631894
0.9370243511171602
0.9366602565291412
0.9363138527003336
0.9359840299777712
0.9356697678003446
0.9353701259227754
0.9350842366580443
0.9348112980029917
0.9345505675319602
0.9343013569601865
0.934063027292768

0.9290872146546069

In [97]:
eigen(m.E + m.F*X)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 0.9
 0.9278097265581992
vectors:
2×2 Matrix{Float64}:
  0.0361552  0.0
 -0.999346   1.0

In [88]:
spectral_radius(u->L_T(u, X, X, m), X)

1.22568251832885
1.0875401310195292
1.0188861995073961
0.9778415851174042
0.9505503157337445
0.931099318948722
0.9165402875827734
0.90523851270917
0.8962146223685403
0.8888460984052489
0.8827183116529246
0.8775444277891336
0.8731196938021573
0.8692940369899395
0.8659549536203057
0.8630164447897879
0.8604116448870422
0.858087780635252
0.8560026441039023
0.8541220744990936
0.8524181274329848
0.8508677222373058
0.8494516277622413
0.8481536918004834
0.8469602484889611
0.8458596575078802
0.844841942101573
0.8438985020493184
0.8430218840848323
0.8422055967828365
0.8414439601787578
0.8407319827489524
0.840065260114708
0.8394398911224966
0.8388524079196674
0.8382997173761858
0.8377790517611281
0.8372879270118609
0.8368241072664376
0.8363855745892895
0.8359705030241966
0.8355772362697191
0.8352042684004926
0.834850227160337
0.834513859435622
0.8341940185840038
0.8338896533478273
0.8335997981256987
0.8333235644119937
0.8330601332439176
0.8328087485204202
0.8325687110777595
0.8323393734235806
0.8

0.8275997629892197

__Write a function `simulate(s0::Vector, X::Matrix, p::Calibration, T::Int64)::Tuple{Matrix, Matrix}` to simulate the model over $T$ periods (by using the formula $\Delta s_t = (E + F X) s_{t-11}$. Return a matrix for the states (one line per date) and another matrix for the controls. Bonus: add a keyword option to compute variables levels or log-deviations. If possible, return a DataFrame object.__

__Make some nice plots.__